# Image classification (MNIST) using Convnets


- [Install PyTorch](https://pytorch.org/get-started/locally/)
- [Original source](https://github.com/pytorch/examples/tree/master/mnist)


In [1]:
from __future__ import print_function

from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() # https://realpython.com/python-super/
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50) # https://stackoverflow.com/a/42482819/1874690
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(data),
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item()
            ))

In [4]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad(): # https://pytorch.org/docs/stable/autograd.html#locally-disable-grad
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss,
        correct,
        len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)
    ))

args:

- batch_size: input batch size for training (default: 64)
- test_batch_size: input batch size for testing (default: 1000)
- epochs: number of epochs to train (default: 10)
- lr: learning rate (default: 0.01)
- momentum: SGD momentum (default: 0.5)
- no_cuda: disables CUDA training
- seed: random seed (default: 1)
- log_interval: how many batches to wait before logging training status (default: 10)
- save_model: For Saving the current Model

In [5]:
def main():
    Arguments = namedtuple("Arguments", [
        "batch_size", 
        "test_batch_size",
        "epochs",
        "lr",
        "momentum",
        "no_cuda",
        "seed",
        "log_interval",
        "save_model"
    ])
    
    args = Arguments(
        batch_size=64, 
        test_batch_size=1000,
        epochs=10,
        lr=0.01,
        momentum=0.5,
        no_cuda=True,
        seed=1,
        log_interval=10,
        save_model=False
    )

    print(args)
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(
            '../data',
            train=True,
            download=True,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ])
        ),
        batch_size=args.batch_size,
        shuffle=True,
        **kwargs
    )

    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(
            '../data',
            train=False,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ])
        ),
        batch_size=args.test_batch_size,
        shuffle=True,
        **kwargs
    )

    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)

    if (args.save_model):
        torch.save(model.state_dict(),"mnist_cnn.pt")

In [6]:
import time

start = time.time()
main()
end = time.time()
print(end - start)

Arguments(batch_size=64, test_batch_size=1000, epochs=10, lr=0.01, momentum=0.5, no_cuda=True, seed=1, log_interval=10, save_model=False)
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300039
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.213470
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.170460
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.076699
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.868078
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.414199
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.000870
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.775736
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.460019
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.486538
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.437759
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.409982
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.462366
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.429528
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.399872
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.387182
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.298461
Train Epoch: 1 [10880/60000 (18%)]	Loss: 

Train Epoch: 2 [43520/60000 (72%)]	Loss: 0.031342
Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.043630
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.037982
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.152209
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.103554
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.133584
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.138977
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.052804
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.053779
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.027299
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.073960
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.100005
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.027172
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.040181
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.023484
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.039920
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.187934
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.063183
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.044461
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.021397


Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.016413
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.043423
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.026739
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.006304
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.059535
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.008481
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.034049
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.014420
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.012474
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.132594
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.058490
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.085103
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.075036
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.031371
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.132453
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.008768
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.032947
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.010155
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.011035
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.051904


Train Epoch: 6 [11520/60000 (19%)]	Loss: 0.035333
Train Epoch: 6 [12160/60000 (20%)]	Loss: 0.017669
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.036987
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.015426
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.028730
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.006841
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.012006
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.015106
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.013537
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.013140
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.052557
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.022896
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.018656
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.033050
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.039346
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.002036
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.059670
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.025930
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.003487
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.018564


Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.045312
Train Epoch: 7 [56960/60000 (95%)]	Loss: 0.003230
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.011518
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.046950
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.013919
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.031305

Test set: Average loss: 0.0345, Accuracy: 9874/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.004609
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.005457
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.007576
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.014307
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.060121
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.031899
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.024039
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.005592
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.003279
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.010553
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.035525
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.003384
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.009262
Train Epoch: 8 

Train Epoch: 9 [40320/60000 (67%)]	Loss: 0.019878
Train Epoch: 9 [40960/60000 (68%)]	Loss: 0.003283
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.007568
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.005100
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.026814
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.006511
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.003080
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.038093
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.020642
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.065830
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.046491
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.000972
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.018784
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.001010
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.020010
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.002404
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.009678
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.005122
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.003964
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.028669
